In [25]:
import matplotlib.pyplot as plt
import ipdb
import os, glob, shutil
import os.path as osp
import numpy as np
import tqdm
from pydicom import dcmread
from PIL import Image

def sorted_list(path): 
    
    """ function for getting list of files or directories. """
    
    tmplist = glob.glob(path) # finding all files or directories and listing them.
    tmplist.sort() # sorting the found list
    
    return tmplist

def crop(ds,patch_size,stride):
    patches = []
    for left in range(0, ds.shape[0] - patch_size + 1, stride):
        for top in range(0, ds.shape[1] - patch_size + 1, stride):
            patches.append(ds[left: left + patch_size, top: top + patch_size])
    return patches


def arr_size(arr,size):
    s=[]
    for i in range(0,int(len(arr))+1,size):
        c=arr[i:i+size]
        s.append(c)
    return s

def make_dir(path, refresh=False):
    
    """ function for making directory (to save results). """
    
    try: os.mkdir(path)
    except: 
        if(refresh): 
            shutil.rmtree(path)
            os.mkdir(path)


In [29]:


data_root='/data/zhchen/aapm_challenge/data2d/'
# patient_root='/data/zhchen/aapm_challenge/data2d/'
# patient_list=sorted_list(patient_root+i+'/*')

#data_split={'train':[0:8],'test':[8:10]}
"""
train:L067,L096,L109,L143,L192,L286,L219,L310
test: L333,L506
"""

dataset_name = '/data/zhchen/Mayo2016_2d'
threshold = 0.85



patch_size=512
stride=512


for phase in ['train','test']:
    full_patches=[]
    quarter_patches=[]
#     save_root_q=dataset_name+'/'+phase+'/'+'quarter_1mm'
#     save_root_f=dataset_name+'/'+phase+'/'+'full_1mm'
#     make_dir(save_root_q, refresh=True)
#     make_dir(save_root_f, refresh=True)
    print('generate {} data...'.format(phase))
    for i in ['full_1mm','quarter_1mm']:
        print('generate {} data...'.format(i))
        if phase=='train':
            patient_list=sorted_list(data_root+'/'+i+'/*')[0:8]
        if phase=='test':
            patient_list=sorted_list(data_root+'/'+i+'/*')[8:10]
        for patient_id,patient_path in enumerate(tqdm.tqdm(patient_list)):
            ct_list = sorted_list(patient_path+'/'+i+'/*')
            for path in ct_list:
                #ipdb.set_trace()
                img=dcmread(path)
                patient_id=img.SliceLocation
                img_array=img.pixel_array.astype(np.float32)
                patches=crop(img_array,patch_size,stride)
                if i =='quarter_1mm':
                    quarter_patches.extend(patches)
                if i =='full_1mm':
                    full_patches.extend(patches)
                    
    print(len(quarter_patches))
    print(len(full_patches))

    count=0
    for k in tqdm.trange((len(full_patches))):
        #print(black_percent)
        black_percent = np.mean(np.clip(full_patches[k] - 1024, -500, 2000) == -500)
        if black_percent < threshold:
            count+=1
            #print(black_percent)
            import ipdb
            #ipdb.set_trace()
#             q_patch = np.array(quarter_patches[k]).astype(np.uint16).reshape(( 1,patch_size, patch_size))
#             f_patch = np.array(full_patches[k]).astype(np.uint16).reshape(( 1,patch_size, patch_size))
#             np.save(save_root_q+'/'+phase+'-quarter_1mm-'+'%05d' % k, np.array(q_patch).astype(np.uint16))
#             np.save(save_root_f+'/'+phase+'-full_1mm-'+'%05d' % k, np.array(f_patch).astype(np.uint16))
    print(count)
    print('complete save {} patches...'.format(phase))
    

    


generate train data...
generate full_1mm data...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.08it/s]


generate quarter_1mm data...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.02it/s]


4800
4800


...

generate quarter_1mm data...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.17it/s]


1136
1136


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1136/1136 [00:01<00:00, 1038.62it/s]

1136
complete save test patches...


In [31]:
'''
ASCON mayo2020
'''
from more_itertools import chunked

np.random.seed(0)


data_path='/data/PUBLIC-DATASET/CT/CT/'

train_list=['L107','L114','L116','L125','L131','L134','L145','L150',
            'L160','L170','L175','L178','L186','L187','L193','L203']

test_list=['L110','L123','L148','L179']

data_lists={'train512':train_list,'test':test_list,'train256':train_list}

dataset_name = '/data/zhchen/Mayo2020_ab_2d'
thresholds = [0.7,0.85]
num_samples = {
    'train512': 2000,
    'train256': 9600,
    'test':600
}
train_patch_size=256
train_stride=64

test_patch_size=512
test_stride=512


for phase in ['train256','test']: 
    full_patches=[]
    quarter_patches=[]
    final_patches=[]
    final_patches1=[]
    final_patches2=[]
#     save_root_q=dataset_name+'/'+phase+'/'+'quarter_1mm'
#     save_root_f=dataset_name+'/'+phase+'/'+'full_1mm'
#     make_dir(save_root_q, refresh=True)
#     make_dir(save_root_f, refresh=True)
    print('generate {} data...'.format(phase))
    for i in range(2):
        print('generate full dose data...') if i==0 else print('generate 25% low dose data...')
        for patient_id in tqdm.tqdm(range(len(data_lists[phase]))):
            dose_list = sorted_list(data_path+data_lists[phase][patient_id]+'/*/*')
            ct_list=sorted_list(dose_list[i]+'/*')
            
            patient_silces=[]
            for path in ct_list:
                img=dcmread(path)
                img_array=img.pixel_array.astype(np.float32)
                patient_silces.append(img_array)
                if 'train' in phase:
                    patches=crop(img_array,train_patch_size,train_stride)
                if 'test' in phase:
                    patches=crop(img_array,test_patch_size,test_stride)

                if i ==1:      
                    quarter_patches.extend(patches)
                if i ==0:
                    full_patches.extend(patches)

#     print(len(full_patches))
#     print(len(quarter_patches))
        
    patch_id = np.random.permutation(len(full_patches))

            
    count=0
    #ipdb.set_trace()
    #for k in tqdm.trange((len(full_patches))):
    for k in tqdm.trange(len(patch_id)):
        if 'train' in phase:
            threshold=thresholds[0]
        if 'test' in phase:
            threshold=thresholds[1] 
        black_percent = np.mean(np.clip(full_patches[k] - 1024, -500, 500) == -500)
        if black_percent < threshold:
            count+=1
            
#             if 'train' in phase:    
#                 q_patch = np.array(quarter_patches[k]).astype(np.uint16).reshape(( 1,train_patch_size, train_patch_size))
#                 f_patch = np.array(full_patches[k]).astype(np.uint16).reshape(( 1,train_patch_size,train_patch_size))
#             if 'test' in phase:
#                 q_patch = np.array(quarter_patches[k]).astype(np.uint16).reshape(( 1,test_patch_size, test_patch_size))
#                 f_patch = np.array(full_patches[k]).astype(np.uint16).reshape(( 1,test_patch_size, test_patch_size))
#             np.save(save_root_q+'/'+phase+'-quarter_1mm-'+'%05d' % k, np.array(q_patch).astype(np.uint16))
#             np.save(save_root_f+'/'+phase+'-full_1mm-'+'%05d' % k, np.array(f_patch).astype(np.uint16))
            #print(k)
        if count==num_samples[phase]:
            break
    print(count)
    print('complete save {} patches...'.format(phase))

    
    


generate train256 data...
generate full dose data...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.70it/s]


generate 25% low dose data...


 16%|█████████████████████████▌                                                                                                                                 | 9730/59025 [00:02<00:12, 3845.81it/s]


9600
complete save train256 patches...
generate test data...
generate full dose data...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.79it/s]


generate 25% low dose data...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 580/580 [00:00<00:00, 1109.20it/s]

580
complete save test patches...
